In [1]:
# let's import some libraries, shall we?

import numpy as np
import pandas as pd

In [2]:
# fetching the csv's into dataframes

df_clients = pd.read_csv("clients.csv")
df_sales = pd.read_csv("ventes.csv")
df_products = pd.read_csv("produits.csv")

In [13]:
# now it's time to clean them a bit

# let's start by removing duplicates
df_clients = df_clients.drop_duplicates()
df_sales = df_sales.drop_duplicates()
df_products = df_products.drop_duplicates()

# print(df_clients.isnull().sum()
# .sort_values(ascending = False) / len(df_clients))
# print(df_sales.isnull().sum()
# .sort_values(ascending = False) / len(df_sales))
# print(df_products.isnull().sum()
# .sort_values(ascending = False) / len(df_products))

# here was checked whether data was missing or not
# and it confirmed there was no missing data detected

In [19]:
print(df_clients.columns)
print(df_sales.columns)
print(df_products.columns)

# while there still are negative prices
while int(df_products[['price']].min()) < 0:
    incorrect_price = df_products['price'].argmin()
    df_products = df_products.drop(
        incorrect_price
    ).reset_index(drop = True) # drop it

Index(['client_id', 'sex', 'birth'], dtype='object')
Index(['id_prod', 'date', 'session_id', 'client_id'], dtype='object')
Index(['id_prod', 'price', 'categ'], dtype='object')
